In [1]:
import sknw
print(sknw.__file__)

/Users/alaink/.local/lib/python3.9/site-packages/sknw-0.13-py3.9.egg/sknw/__init__.py


In [9]:
import cv2 as cv

img = cv.imread('TestData/AgNWN_10um/Binarized/slice0.tiff', cv.IMREAD_GRAYSCALE)
#plt.imshow(img)
canvas = np.ones(img.shape)
new_canvas = canvas
new_canvas[disk_pos[0], disk_pos[1]] = 0
circle = np.ma.MaskedArray(img, mask=new_canvas)
plt.imshow(circle)

f.img_bin.shape

NameError: name 'disk_pos' is not defined

In [2]:
#Draft
import numpy as np
import igraph as ig
import os
import cv2 as cv
import base
import process_image
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import error
import time
import functools
import gsd.hoomd
from skimage.morphology import skeletonize_3d, disk
import pandas as pd
import itertools

class Network():
    """Generic SGT graph class: a specialised case of the igraph Graph object with 
    additional attributes defining geometric features, associated images,
    dimensionality etc.
    
    Initialised from directory containing raw image data
    self._2d determined from the number of images with identical dimensions (suggesting a stack when > 1)
    
    Image shrinking/cropping is carried out at the gsd stage in analysis.
    I.e. full images are binarized but cropping their graphs may come after
    """
    def __init__(self, directory, child_dir='/Binarized'):
        if not isinstance(directory, str):
            raise TypeError
            
        self.dir = directory
        self.child_dir = child_dir
        self.stack_dir = self.dir + self.child_dir
        
        shape = []
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            shape.append(cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE).shape)
        
        if len(set(shape)) == len(shape):
            self._2d = True
            self.dim = 2
        else:
            self._2d = False
            self.dim = 3
        
    def binarize(self, options_dict=None):
        """Binarizes stack of experimental images using a set of image processing parameters in options_dict.
        Note this enforces that all images have the same shape as the first image encountered by the for loop.
        (i.e. the first alphanumeric titled image file)
        """
        
        if options_dict is None:
            options = self.dir + '/img_options.json'
            with open(options) as f:
                options_dict = json.load(f)

        if not os.path.isdir(self.dir + self.child_dir):
            os.mkdir(self.dir + self.child_dir)

        i=0
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            else:
                img_exp = cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE)
                if i == 0: shape = img_exp.shape
                elif img_exp.shape != shape: continue
                _, img_bin, _ = process_image.binarize(img_exp, options_dict)
                #_, img_bin, _ = process_image.binarize(img_bin, options_dict)
                #_, img_bin, _ = process_image.binarize(img_bin, options_dict)
                #_, img_bin, _ = process_image.binarize(img_bin, options_dict)
                #plt.imshow(img_bin, cmap=cm.gray)
                plt.imsave(self.dir+self.child_dir+'/slice'+str(i)+'.tiff', img_bin, cmap=cm.gray)
                i+=1
        
    def stack_to_gsd(self, name='skel.gsd', crop=None, skeleton=True, rotate=None, debubble=None):
        """Writes a .gsd file from the object's directory.
        The name of the written .gsd is set as an attribute so it may be easily matched with its Graph object 
        Running this also sets the positions, shape attributes
        
        For 2D graphs, the first element of the 3D position list is all 0s. So the gsd y corresponds to the graph
        x and the gsd z corresponds to the graph y.

        """
        start = time.time()
        self.gsd_name = self.stack_dir + '/' + name
        self.gsd_dir = os.path.split(self.gsd_name)[0]
        img_bin=[]

        #Initilise i such that it starts at the lowest number belonging to the images in the stack_dir
        #First require boolean mask to filter out non image files
        olist = np.asarray(sorted(os.listdir(self.stack_dir)))
        mask = list(base.Q_img(olist[i]) for i in range(len(olist)))
        if len(mask) == 0:
            raise error.ImageDirectoryError(self.stack_dir)
        fname = sorted(olist[mask])[0] #First name
        i = int(os.path.splitext(fname)[0][5:]) #Strip file type and 'slice' then convert to int

        #Generate 3d (or 2d) array from stack
        for fname in sorted(os.listdir(self.stack_dir)):
            if base.Q_img(fname):
                img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
                if rotate is not None:
                    image_center = tuple(np.array(img_slice.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(image_center, rotate, 1.0)
                    img_slice = cv.warpAffine(img_slice, rot_mat, img_slice.shape[1::-1], flags=cv.INTER_LINEAR)
                img_bin.append(img_slice)
                i=i+1
            else:
                continue

        #For 2D images, img_bin_3d.shape[0] == 1
        img_bin = np.asarray(img_bin)
        self.img_bin_3d = img_bin
        self.img_bin = img_bin

        #Note that numpy array slicing operations are carried out in reverse order!
        #(...hence crop 2 and 3 before 0 and 1)
        if crop and self._2d:
            self.img_bin = self.img_bin[:, crop[2]:crop[3], crop[0]:crop[1]]
            #img_bin = img_bin[crop[0]:crop[1], crop[2]:crop[3]]
        elif crop:
            #TODO figure tf this bit out
            self.img_bin = self.img_bin[crop[0]:crop[1], crop[2]:crop[3], crop[4]:crop[5]]

        assert self.img_bin_3d.shape[1] > 1
        assert self.img_bin_3d.shape[2] > 1
        
        self.img_bin_3d = self.img_bin            #Always 3d, even for 2d images
        self.img_bin = np.squeeze(self.img_bin)   #3d for 3d images, 2d otherwise

        assert self.img_bin_3d.shape[1] == self.img_bin.shape[0]
        assert self.img_bin_3d.shape[2] == self.img_bin.shape[1]
        
        if skeleton:
            self.skeleton = skeletonize_3d(np.asarray(self.img_bin))
            self.skeleton_3d = skeletonize_3d(np.asarray(self.img_bin_3d))
            #self.skeleton_3d = np.swapaxes(self.skeleton_3d, 1, 2)
        else:
            self.img_bin = np.asarray(self.img_bin)

        positions = np.asarray(np.where(np.asarray(self.skeleton_3d) == 255)).T
        self.shape = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)[0:self.dim]))
        self.positions = positions

        print(positions)
        print(self.img_bin.shape)

        with gsd.hoomd.open(name=self.gsd_name, mode='wb') as f:
            s = gsd.hoomd.Snapshot()
            s.particles.N = len(positions)
            s.particles.position = base.shift(positions)
            s.particles.types = ['A']
            s.particles.typeid = ['0']*s.particles.N
            f.append(s)

        end = time.time()
        print('Ran stack_to_gsd() in ', end-start, 'for gsd with ', len(positions), 'particles')

        if debubble is not None: self = base.debubble(self, debubble)

        assert self.img_bin.shape == self.skeleton.shape
        assert self.img_bin_3d.shape == self.skeleton_3d.shape

        
    def stack_to_circular_gsd(self, radius, name='circle.gsd', rotate=None, debubble=None, skeleton=True):
        """Writes a cicular .gsd file from the object's directory.
        Currently only capable of 2D graphs
        Unlike stack_to_gsd, the axis of rotation is not the centre of the image, but the point (radius,radius)
        The name of the written .gsd is set as an attribute so it may be easily matched with its Graph object 
        Running this also sets the positions, shape attributes
        """
        start = time.time()
        self.gsd_name = self.stack_dir + '/' + name
        self.gsd_dir = os.path.split(self.gsd_name)[0]
        img_bin=[]
        
        #Initilise i such that it starts at the lowest number belonging to the images in the stack_dir
        #First require boolean mask to filter out non image files
        olist = np.asarray(sorted(os.listdir(self.stack_dir)))
        mask = list(base.Q_img(olist[i]) for i in range(len(olist)))
        if len(mask) == 0:
            raise error.ImageDirectoryError(self.stack_dir)
        fname = sorted(olist[mask])[0] #First name
        i = int(os.path.splitext(fname)[0][5:]) #Strip file type and 'slice' then convert to int
        
        img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
        
        #Read the image
        for fname in sorted(os.listdir(self.stack_dir)):
            if base.Q_img(fname):
                img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
                if rotate is not None:
                    axis_of_rot = tuple((radius,radius))
                    #image_center = tuple(np.array(img_slice.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(axis_of_rot, rotate, 1.0)
                    img_slice = cv.warpAffine(img_slice, rot_mat, img_slice.shape[1::-1], flags=cv.INTER_LINEAR)
                img_bin.append(img_slice)
                i=i+1
            else:
                continue
                
        #For 2D images, img_bin_3d.shape[0] == 1
        img_bin = np.asarray(img_bin)
        
        self.img_bin_3d = img_bin            #Always 3d, even for 2d images
        self.img_bin = np.squeeze(img_bin)   #3d for 3d images, 2d otherwise
        
        #Note that numpy array slicing operations are carried out in reverse order!
        #(...hence crop 2 and 3 before 0 and 1)
        assert self._2d


        canvas = np.ones(self.img_bin.shape)
        disk_pos = np.asarray(np.where(disk(radius)!=0)).T
        canvas[disk_pos[0], disk_pos[1]] = 0
        self.img_bin = np.ma.MaskedArray(self.img_bin, mask=canvas)
        self.img_bin = np.ma.filled(self.img_bin, fill_value=0)
        
        canvas = np.ones(self.img_bin_3d.shape)
        disk_pos = np.asarray(np.where(disk(radius)!=0)).T
        disk_pos = np.array([np.zeros(len(disk_pos)), disk_pos.T[0], disk_pos.T[1]], dtype=int)
        canvas[disk_pos[0], disk_pos[1], disk_pos[2]] = 0
        self.img_bin_3d = np.ma.MaskedArray(self.img_bin_3d, mask=canvas)
        self.img_bin_3d = np.ma.filled(self.img_bin_3d, fill_value=0)
        self.img_bin = self.img_bin_3d[0]
        
        assert self.img_bin_3d.shape[1] > 1
        assert self.img_bin_3d.shape[2] > 1
        
        if skeleton:
            self.skeleton = skeletonize_3d(np.asarray(self.img_bin))
            self.skeleton_3d = skeletonize_3d(np.asarray(self.img_bin_3d))
        else:
            self.img_bin = np.asarray(self.img_bin)
        
        positions = np.asarray(np.where(np.asarray(self.skeleton_3d) == 255)).T
        self.shape = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)[0:self.dim]))
        self.positions = positions
        
        with gsd.hoomd.open(name=self.gsd_name, mode='wb') as f:
            s = gsd.hoomd.Snapshot()
            s.particles.N = len(positions)
            s.particles.position = base.shift(positions)
            s.particles.types = ['A']
            s.particles.typeid = ['0']*s.particles.N
            f.append(s)
        
        end = time.time()
        print('Ran stack_to_gsd() in ', end-start, 'for gsd with ', len(positions), 'particles')
        
        if debubble is not None: self = base.debubble(self, debubble)
            
        assert self.img_bin.shape == self.skeleton.shape
        assert self.img_bin_3d.shape == self.skeleton_3d.shape    
        
        
    def G_u(self):
        """Sets unweighted igraph object as an attribute
        """
        G =  base.gsd_to_G(self.gsd_name, _2d = self._2d)
        self.Gr = G
        self.shape = list(max(list(self.Gr.vs[i]['o'][j] for i in range(self.Gr.vcount()))) for j in (0,1,2)[0:self.dim])
        
    def weighted_Laplacian(self):

        L=np.asarray(self.Gr.laplacian(weights='weight'))
        self.L = L

class ResistiveNetwork(Network):
    """Child of generic SGT Network class.
    Equipped with methods for analysing resistive flow networks

    """
    def __init__(self, directory):
        super().__init__(directory)
        
    def potential_distribution(self, plane, boundary1, boundary2, R_j=0, rho_dim=1, F_dim=1):
        """Solves for the potential distribution in a weighted network.
        Source and sink nodes are connected according to a penetration boundary condition.
        Sets the corresponding weighted Laplacian, potential and flow attributes.
        The 'plane' arguement defines the axis along which the boundary arguements refer to.
        R_j='infinity' enables the unusual case of all edges having the same unit resistance.
        
        NOTE: Critical that self.G_u() is called before every self.potential_distribution()
        TODO: Remove this requirement or add an error to warn
        """
        self.Gr = base.sub_G(self.Gr)
        print('post sub has ', self.Gr.vcount(), ' nodes')
        if R_j != 'infinity':
            print(self.Gr.vcount())
            self.Gr_connected = base.add_weights(self, weight_type='Resistance', R_j=R_j, rho_dim=rho_dim)
            print(self.Gr.vcount())
            weight_array = np.asarray(self.Gr_connected.es['weight']).astype(float)
            weight_array = weight_array[~np.isnan(weight_array)]
            self.edge_weights = weight_array
            weight_avg =np.mean(weight_array)
        else:
            self.Gr_connected.es['weight'] = np.ones(self.Gr_connected.ecount())
            weight_avg = 1

        #Add source and sink nodes:
        source_id = max(self.Gr_connected.vs).index + 1
        sink_id = source_id + 1
        self.Gr_connected.add_vertices(2)

        print('Graph has max ', self.shape)
        axes = np.array([0,1,2])[0:self.dim]
        indices = axes[axes!=plane]
        plane_centre1 = np.zeros(self.dim, dtype=int)
        delta = np.zeros(self.dim, dtype=int)
        delta[plane] = 10 #Arbitrary. Standardize?
        for i in indices: plane_centre1[i] = self.shape[i]/2
        plane_centre2 = np.copy(plane_centre1)
        plane_centre2[plane] = self.shape[plane]
        source_coord = plane_centre1 - delta 
        sink_coord = plane_centre2 + delta
        print('source coord is ', source_coord)
        print('sink coord is ', sink_coord)
        self.Gr_connected.vs[source_id]['o'] = source_coord
        self.Gr_connected.vs[sink_id]['o'] = sink_coord

    #Connect nodes on a given boundary to the external current nodes
        print('Before connecting external nodes, G has vcount ', self.Gr_connected.vcount())
        for node in self.Gr_connected.vs:
            if node['o'][plane] > boundary1[0] and node['o'][plane] < boundary1[1]:
                self.Gr_connected.add_edges([(node.index, source_id)])
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,source_id)]['weight'] = weight_avg
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,source_id)]['pts'] = base.connector(source_coord,node['o'])
            if node['o'][plane] > boundary2[0] and node['o'][plane] < boundary2[1]:
                self.Gr_connected.add_edges([(node.index, sink_id)])
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,sink_id)]['weight'] = weight_avg 
                self.Gr_connected.es[self.Gr_connected.get_eid(node.index,sink_id)]['pts'] = base.connector(sink_coord,node['o'])

    #Write skeleton connected to external node
        print(self.Gr_connected.is_connected(), ' connected')
        print('After connecting external nodes, G has vcount ', self.Gr_connected.vcount())
        connected_name = os.path.split(self.gsd_name)[0] + '/connected_' + os.path.split(self.gsd_name)[1] 
        #connected_name = self.stack_dir + '/connected_' + self.gsd_name 
        base.G_to_gsd(self.Gr_connected, connected_name)

        self.weighted_Laplacian()
        F = np.zeros(sink_id+1)
        print(F.shape,'F')
        print(self.L.shape, 'L')
        F[source_id] = F_dim
        F[sink_id] = -F_dim
        np.save(self.stack_dir+'/L.npy',self.L)
        np.save(self.stack_dir+'/F.npy',F)
        P = np.matmul(np.linalg.pinv(self.L, hermitian=True),F)
        np.save(self.stack_dir+'/P.npy',P)

        self.P = P
        self.F = F

class StructuralNetwork(Network):
    """Child of generic SGT Network class.
    Equipped with methods for analysing structural networks
    """
    def __init__(self, directory):
        super().__init__(directory)
        
    def G_calc(self):
        avg_indices = dict()

        operations = [self.Gr.diameter, self.Gr.density, self.Gr.transitivity_undirected, self.Gr.assortativity_degree]
        names = ['Diameter', 'Density', 'Clustering', 'Assortativity by degree']

        for operation,name in zip(operations,names):
            start = time.time()
            avg_indices[name] = operation()
            end = time.time()
            print('Calculated ', name, ' in ', end-start)
            
        self.G_attributes = avg_indices
        
    def node_calc(self):
        self.Betweenness = self.Gr.betweenness()
        self.Closeness = self.Gr.closeness()
        self.Degree = self.Gr.degree()

def Node_labelling(g, attribute, attribute_name, filename):
    """
    Function saves a new .gsd which has the graph in g.Gr labelled with the node attributes in attribute
    """
    
    assert g.Gr.vcount() == len(attribute)
    
    #save_name = os.path.split(prefix)[0] + '/'+attribute_name + os.path.split(prefix)[1]
    save_name = g.dir + '/' + filename
    if os.path.exists(save_name):
        mode = 'rb+'
    else:
        mode = 'wb'

    f = gsd.hoomd.open(name=save_name, mode=mode)
    
    #Must segregate position list into a node_position section and edge_position
    node_positions = np.asarray(list(g.Gr.vs()[i]['o'] for i in range(g.Gr.vcount())))
    positions = node_positions
    for edge in g.Gr.es():
        positions=np.vstack((positions,edge['pts']))
    positions = np.unique(positions, axis=0)
    if g._2d:
        node_positions = np.hstack((np.zeros((len(node_positions),1)),node_positions))
        positions = np.hstack((np.zeros((len(positions),1)),positions))

    #node_positions = base.shift(node_positions)
    #positions = base.shift(positions)
    s = gsd.hoomd.Snapshot()
    N = len(positions)
    s.particles.N = N
    s.particles.position = positions
    s.particles.types = ['Edge', 'Node']
    s.particles.typeid = [0]*N
    L = list(max(positions.T[i])*2 for i in (0,1,2))
    #s.configuration.box = [L[0]/2, L[1]/2, L[2]/2, 0, 0, 0]
    s.configuration.box = [1,1,1,0,0,0]
    s.log['particles/'+attribute_name] = [np.NaN]*N
    start = time.time()

    j=0
    for i,particle in enumerate(positions):
        node_id = np.where(np.all(positions[i] == node_positions, axis=1) == True)[0]
        if len(node_id) == 0: 
            continue
        else:
            s.log['particles/'+attribute_name][i] = attribute[node_id[0]]
            s.particles.typeid[i] = 1
            j+=1
    
    f.append(s)

In [75]:
scale=1
img_options={"Thresh_method":1, "gamma": 2.34, "md_filter": 0, "g_blur": 1, "autolvl": 0,
             "fg_color":1, "laplacian": 0, "scharr": 1, "sobel":0 , "lowpass": 1, "asize": int((103*scale))*2+1,
             "bsize":int((43*scale))*2+1, "wsize": int((10*scale))*2+1, "thresh": 127}


"""
img_options={"Thresh_method":1, "gamma": 2.34, "md_filter": 0, "g_blur": 1, "autolvl": 0,
             "fg_color":1, "laplacian": 0, "scharr": 1, "sobel":1 , "lowpass": 1, "asize": 103,
             "bsize":43, "wsize": 10, "thresh": 127}
"""
f = ResistiveNetwork('TestData/Mxene/1')
#f.binarize(options_dict=img_options)
f.stack_to_gsd(debubble=[disk(5)])

[[   0    0   50]
 [   0    0   54]
 [   0    0  144]
 ...
 [   0 1756 1907]
 [   0 1756 1916]
 [   0 1756 1963]]
(1757, 2033)
Ran stack_to_gsd() in  4.061809062957764 for gsd with  239234 particles
Ran debubble in  3.6325650215148926 for an image with shape  (1, 1757, 2033)


In [33]:
"""Examples:
f = ResistiveNetwork('TestData/AgNWN_10um')
f.binarize()
f.stack_to_gsd(crop=[300,600,0,500], debubble=[disk(3)])
f.G_u()
f.potential_distribution(1, [0,20], [480,500])
base.Node_labelling(f, f.P, 'P', 'test1.gsd')

g = ResistiveNetwork('TestData/pores-hi-res-crop')
g.stack_to_gsd(crop=[0,100,0,100,0,100], debubble=[ball(3)])
g.G_u()
g.potential_distribution(0, [0,20], [80,100])
base.Node_labelling(g, g.P, 'P', 'test2.gsd')

s = StructuralNetwork('TestData/fibres-hi-res-crop')
s.stack_to_gsd(crop=[0,100,0,100,0,100])
s.G_u()
s.node_calc()
base.Node_labelling(s, s.Degree, 'Degree', 'test3.gsd')
"""

f = ResistiveNetwork('TestData/AgNWN_10um')
f.stack_to_circular_gsd(400, rotate=45, debubble=[disk(1)])
f.G_u()
f.potential_distribution(1, [0,50], [750,800])
Node_labelling(f, f.P, 'P', 'test1.gsd')

Ran stack_to_gsd() in  0.4855809211730957 for gsd with  57680 particles
Ran debubble in  0.4514319896697998 for an image with shape  (1, 1018, 1536)
gsd_to_G canvas has shape  (799, 799)
[[  3 361]]
(0, 30, array([[  3, 361],
       [  4, 360],
       [  5, 359],
       [  6, 358],
       [  7, 358],
       [  8, 357],
       [  9, 356],
       [ 10, 355],
       [ 11, 354],
       [ 12, 353],
       [ 13, 352],
       [ 14, 351],
       [ 15, 351],
       [ 16, 351],
       [ 17, 351],
       [ 18, 351],
       [ 19, 352],
       [ 20, 353]], dtype=int16))
Ran gsd_to_G in  2.523740768432617 for a graph with  4270 nodes.
pre sub has  4270  nodes
post sub has  4043  nodes
post sub has  4043  nodes
4043
4043
Graph has max  [798, 797]
source coord is  [399 -10]
sink coord is  [399 807]
Before connecting external nodes, G has vcount  4045
True  connected
After connecting external nodes, G has vcount  4045
(4045,) F
(4045, 4045) L


implicit data copy when writing chunk: log/particles/P


In [5]:
#Rotation test
#Continuous rotation analysis
#Testing base.py implementation
#This implementation rotates the image itself

#######################################################
## USE THIS CELL FOR TESTING THE ABOVE DECLARATIONS ##
## USE A SEPARATE NOTEBOOK FOR TESTING network.py   ##
#######################################################

from PIL import Image
from skimage.morphology import binary_closing, disk, ball, skeletonize_3d
import network
g = network.ResistiveNetwork('TestData/AgNWN_10um')
g.binarize()

#thetas = np.linspace(0.001,350, 4)
thetas = (0,45)

#crop = 359

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(g.stack_dir+'/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = int((long_length - ISS)/2)
L3 = int(ISS+L1)
L2 = int((short_length - ISS)/2)
L4 = int(ISS+L2)
o_corn_crop = [L2,L1,L4,L3]
dims = L2-L1



R_j = 0
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
nodes = []
for theta in thetas:
    g.stack_to_gsd(name='Rotations/ObjectImplementation/rot_skel.gsd', crop=[L1,L3,L2,L4], rotate=theta, debubble=[disk(2)])
    g.G_u()
    measuring_graph = base.add_weights(g, weight_type='Resistance', R_j=R_j)
    g.potential_distribution(0, [0,20], [727-20,727], R_j=R_j, rho_dim=0.89, F_dim=1)
    base.Node_labelling(g, g.P, 'P', 'take2.gsd')
    print(g.Gr_connected.vcount())
    print(measuring_graph.vcount())
    
    #Ax, Ay = base.gyration_moments(g.Gr, sampling = 0.1)
    #L = g.L
    #Q = np.linalg.pinv(L)
    #O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    nodes.append(g.Gr.vcount())
    df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
    df_cont['O_eff'] = O_eff_df
    df_cont['Ax'] = Ax_df
    df_cont['Ay'] = Ay_df
    df_cont['Theta'] = theta_df
    #df_cont.to_csv('AgNWN.csv')

    df_cont
    #print(theta, O_eff, Ax, Ay)
    
    #stop
    
    
    #O_eff_df.append(O_eff)
    #Ax_df.append(Ax)
    #Ay_df.append(Ay)
    #theta_df.append(theta)



[[  0   0   4]
 [  0   0   5]
 [  0   0   6]
 ...
 [  0 718 673]
 [  0 718 674]
 [  0 718 675]]
(719, 719)
Ran stack_to_gsd() in  0.24706196784973145 for gsd with  59554 particles
Ran debubble in  0.27390527725219727 for an image with shape  (1, 719, 719)
gsd_to_G canvas has shape  (719, 719)
[[ 0 41]]
(0, 14, array([[ 0, 41],
       [ 0, 42],
       [ 0, 43],
       [ 0, 44],
       [ 0, 45],
       [ 0, 46],
       [ 0, 47],
       [ 0, 48],
       [ 0, 49],
       [ 0, 50],
       [ 1, 51],
       [ 1, 52],
       [ 1, 53],
       [ 1, 54],
       [ 1, 55],
       [ 1, 56],
       [ 1, 57],
       [ 1, 58],
       [ 1, 59],
       [ 1, 60],
       [ 1, 61],
       [ 1, 62],
       [ 1, 63],
       [ 1, 64],
       [ 1, 65],
       [ 1, 66],
       [ 1, 67],
       [ 1, 68],
       [ 1, 69],
       [ 1, 70],
       [ 1, 71],
       [ 1, 72],
       [ 1, 73],
       [ 2, 74],
       [ 2, 75],
       [ 3, 76],
       [ 4, 77],
       [ 4, 78],
       [ 4, 79],
       [ 3, 80],
       [

implicit data copy when writing chunk: log/particles/P


2275
2653
[[  0   0  14]
 [  0   0  27]
 [  0   0 199]
 ...
 [  0 718 659]
 [  0 718 660]
 [  0 718 671]]
(719, 719)
Ran stack_to_gsd() in  0.4410579204559326 for gsd with  57962 particles
Ran debubble in  0.3710489273071289 for an image with shape  (1, 719, 719)
gsd_to_G canvas has shape  (719, 719)
[[1 0]]
(1, 63, array([[ 1, 68],
       [ 1, 67],
       [ 1, 66],
       [ 1, 65],
       [ 1, 64],
       [ 1, 63],
       [ 1, 62],
       [ 2, 61],
       [ 2, 60],
       [ 3, 59],
       [ 3, 58],
       [ 3, 57],
       [ 3, 56],
       [ 4, 55],
       [ 4, 54],
       [ 4, 53],
       [ 5, 52],
       [ 6, 51],
       [ 7, 50],
       [ 7, 49],
       [ 8, 48],
       [ 9, 47],
       [ 9, 46],
       [ 9, 45],
       [10, 44],
       [11, 43],
       [12, 42]], dtype=int16))
Ran gsd_to_G in  2.656190872192383 for a graph with  3135 nodes.


/Users/alaink/.local/lib/python3.9/site-packages/StructuralGT-1.0.1b2-py3.9.egg/StructuralGT/GetWeights_3d.py:32: RuntimeWarning: invalid value encountered in true_divide


pre sub has  3135  nodes
post sub has  2888  nodes
post sub has  2888  nodes
2888
2888
Graph has max  [718, 718]
source coord is  [-10 359]
sink coord is  [728 359]
Before connecting external nodes, G has vcount  2890
True  connected
After connecting external nodes, G has vcount  2890
(2890,) F
(2890, 2890) L


implicit data copy when writing chunk: log/particles/P


2890
3135


In [ ]:
len(np.asarray(np.where(np.asarray(g.img_bin_3d)!=0)).T)

In [37]:
f.edge_weights

array([18.        ,  0.6       ,  0.06      , ...,  0.66666667,
       10.        ,  0.02366864])

In [ ]:
g.img_bin_3d
img_bin = np.swapaxes(g.img_bin_3d, 0, 2)

crop = o_corn_crop
img_bin[crop[2]:crop[3], crop[0]:crop[1]].shape

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(thetas,nodes)
plt.show()

In [ ]:

img_bin=[]
img_bin.append(img_slice)
img_bin = np.asarray(img_bin)
img_bin = np.swapaxes(img_bin, 0, 2)
print(img_bin.shape)
img_bin = img_bin[crop[2]:crop[3],crop[0]:crop[1]]
print(img_bin.shape)

In [ ]:
def timer(_class,method):
    """Print the runtime of the decorated function"""
    @functools.wraps(method)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = _method(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {method.__name__!r} in {run_time:.4f} secs")
        print(_class)
        print(method)
        return value
    return wrapper_timer